In [1]:
import os
import sys

import numpy as np
import pandas as pd
import scipy as sp

import seaborn as sns
import matplotlib.pyplot as plt

PROJ_ROOT =  os.path.join(os.pardir)

print(os.path.abspath(PROJ_ROOT))

d:\DANIELA\Projects\WIDS\covid-outcomes-research


In [2]:
#Let your audience know when this notebook was run, and with which packages. Useful when you are not sharing the notebook as
#executable code.

%pip install watermark

Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext watermark

In [4]:
%watermark -a "Daniela Alvarez Zegarra" -d -t -v -p numpy,pandas

Author: Daniela Alvarez Zegarra

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.3.0

numpy : 1.21.5
pandas: 1.4.2



In [5]:
#Documentation to know what other info can i get with watermark

%watermark?

Docstring:
::

  %watermark [-a AUTHOR] [-gu GITHUB_USERNAME] [-e EMAIL] [-ws WEBSITE]
                 [-d] [-n] [-t] [-i] [-z] [-u] [-c CUSTOM_TIME] [-v]
                 [-p PACKAGES] [-co] [-h] [-m] [-g] [-r] [-b] [-w] [-iv]

IPython magic function to print date/time stamps
and various system information.

optional arguments:
  -a AUTHOR, --author AUTHOR
                        prints author name
  -gu GITHUB_USERNAME, --github_username GITHUB_USERNAME
                        prints author github username
  -e EMAIL, --email EMAIL
                        prints author email
  -ws WEBSITE, --website WEBSITE
                        prints author or project website
  -d, --date            prints current date as YYYY-mm-dd
  -n, --datename        prints date with abbrv. day and month names
  -t, --time            prints current time as HH-MM-SS
  -i, --iso8601         prints the combined date and time including the time
                        zone in the ISO 8601 standard with UTC off

In [6]:
#Let's print our requirements

print(open(os.path.join(PROJ_ROOT,'requirements.txt')).read())

# local package
-e .

# external requirements
click
Sphinx
coverage
awscli
flake8
python-dotenv>=0.5.1
jupyter
ipython
numpy
pandas
matplotlib
scikit-learn
scipy
pytest
nbdime
runipy
seaborn



## Loading the Data

In [7]:
wids_path = os.path.join(PROJ_ROOT,
                          "data",
                          "raw",
                          "dataset.csv")

deaths_path = os.path.join(PROJ_ROOT,
                          "data",
                          "raw",
                          "county_deaths_and_cases_jan_to_may_2020.csv")

medical_conditions_path = os.path.join(PROJ_ROOT,
                          "data",
                          "raw",
                          "medical_conditions_risky_for_covid_by_county.csv")

masterdata_path = os.path.join(PROJ_ROOT,
                          "data",
                          "MasterData.csv")


In [8]:
#Loading main dataset

df = pd.read_csv(wids_path)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 93 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   Unnamed: 0                                                                     3140 non-null   int64  
 1   fips                                                                           3140 non-null   int64  
 2   TOT_POP                                                                        3140 non-null   int64  
 3   0-9                                                                            3140 non-null   int64  
 4   0-9 y/o % of total pop                                                         3140 non-null   float64
 5   10-19                                                                          3140 non-null   int64  
 6   10-19 y/o % of total pop

In [10]:
#We will drop all columns related to temperature because we don't find them relevant

df = df.drop(df.iloc[:, 63:77],axis = 1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 79 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   Unnamed: 0                                                                     3140 non-null   int64  
 1   fips                                                                           3140 non-null   int64  
 2   TOT_POP                                                                        3140 non-null   int64  
 3   0-9                                                                            3140 non-null   int64  
 4   0-9 y/o % of total pop                                                         3140 non-null   float64
 5   10-19                                                                          3140 non-null   int64  
 6   10-19 y/o % of total pop

In [12]:
df.head()

,Unnamed: 0,fips,TOT_POP,0-9,0-9 y/o % of total pop,10-19,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,...,Total nurse practitioners (2019),Total physician assistants (2019),Total Hospitals (2019),Internal Medicine Primary Care (2019),Family Medicine/General Practice Primary Care (2019),Total Specialist Physicians (2019),ICU Beds_x,Total Population,Population Aged 60+,Percent of Population Aged 60+
0,0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,...,28.859137,6.085786,1.148905,25.992561,21.249061,72.142154,6.0,55036,10523,19.1
1,1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,...,113.162114,23.863512,4.505074,101.921730,83.321572,282.882982,51.0,203360,53519,26.3
2,2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,...,12.914231,2.723340,0.514126,11.631462,9.508784,32.283033,5.0,26201,6150,23.5
3,3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,...,11.626493,2.451783,0.462860,10.471635,8.560619,29.063942,0.0,22580,4773,21.1
4,4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,...,30.021267,6.330854,1.195171,27.039257,22.104740,75.047251,6.0,57667,13600,23.6


In [13]:
perc_cols_index = [4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,78]

In [14]:
#Let's see some statistics of the columns expressed in percentage for Ages and Races.

df.iloc[:,perc_cols_index].describe()

,0-9 y/o % of total pop,10-19 y/o % of total pop,20-29 y/o % of total pop,30-39 y/o % of total pop,40-49 y/o % of total pop,50-59 y/o % of total pop,60-69 y/o % of total pop,70-79 y/o % of total pop,80+ y/o % of total pop,% White-alone,% Black-alone,% NA/AI-alone,% Asian-alone,% Hawaiian/PI-alone,% Two or more races,Percent of Population Aged 60+
count,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000
mean,11.871051,12.694609,12.283979,11.751535,11.556685,13.526733,13.182225,8.399382,4.733801,84.494109,9.342405,2.344741,1.554548,0.141007,2.123190,24.736943
std,2.124081,1.815044,3.126297,1.696599,1.372935,1.481692,2.532498,2.214915,1.535028,16.348987,14.478294,7.721990,2.857088,0.967946,1.544463,5.564871
min,0.000000,0.000000,0.000000,6.092789,2.631579,5.316361,3.444403,2.199551,0.000000,3.841985,0.000000,0.000000,0.000000,0.000000,0.000000,5.800000
25%,10.594639,11.674504,10.496774,10.689322,10.685197,12.738561,11.621232,7.032275,3.738648,79.620473,0.867069,0.384294,0.465659,0.031896,1.357557,21.200000
50%,11.802727,12.687422,11.772649,11.580861,11.618372,13.621339,13.013409,8.208162,4.565338,91.410189,2.545440,0.641143,0.739372,0.062024,1.789720,24.400000
75%,12.951840,13.659282,13.182260,12.639379,12.386083,14.429115,14.467254,9.445777,5.484143,95.521727,10.852053,1.331416,1.433411,0.118429,2.409970,27.800000
max,25.460677,23.304372,37.570198,22.225129,19.430369,20.454545,27.272727,31.327959,22.727273,99.043785,86.069762,92.515200,42.952310,48.863636,30.304056,64.200000


We can see that the median and the mean in ages are almost the same for all counties.

In [15]:
# drop all % columns. We will work with real numbers

cols_drop_df = [0,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,78]

In [16]:
new_df = df.drop(df.iloc[:,cols_drop_df], axis= 1)

In [1]:
new_df.to_csv('../data/interim/wids_clean.csv')

NameError: name 'new_df' is not defined

## Visualizing different datasets

### DF2 = DEATHS AND CASES BY MAY 2020

In [17]:
df2 = pd.read_csv(deaths_path)

In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141403 entries, 0 to 141402
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    141403 non-null  object 
 1   county  141403 non-null  object 
 2   state   141403 non-null  object 
 3   fips    139802 non-null  float64
 4   cases   141403 non-null  int64  
 5   deaths  141403 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 6.5+ MB


In [19]:
df2.tail(4)


,date,county,state,fips,cases,deaths
141399,2020-05-13,Sweetwater,Wyoming,56037.0,20,0
141400,2020-05-13,Teton,Wyoming,56039.0,99,1
141401,2020-05-13,Uinta,Wyoming,56041.0,9,0
141402,2020-05-13,Washakie,Wyoming,56043.0,7,0


In [20]:
# Putting sum of deaths/cases per fips code into a variable

deaths_may_2020 = df2.groupby(['fips'])[['deaths']].sum()
cases_may_2020 = df2.groupby(['fips'])[['cases']].sum()

In [21]:
#Convert deaths_may_2020/cases_may_2020 variable into dataframes

deaths_df = pd.DataFrame(data=deaths_may_2020)
cases_df = pd.DataFrame(data=cases_may_2020)

In [22]:
#Merge our two new dataframes, about sum of cases and sum of deaths by May 2020

sums = cases_df.merge(deaths_df, on='fips', how='inner')

In [23]:
sums.head()

,cases,deaths
fips,,
1001.0,1611,90
1003.0,5621,124
1005.0,1193,15
1007.0,1234,6
1009.0,1171,0


We'll save this dataframe to be merged with the main dataset later.

### DF3 : MEDICAL CONDITIONS

In [24]:
df3 = pd.read_csv(medical_conditions_path)

In [25]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COUNTY_NAME                  3142 non-null   object 
 1   STATE_NAME                   3142 non-null   object 
 2   STATE_FIPS                   3142 non-null   int64  
 3   CNTY_FIPS                    3142 non-null   int64  
 4   FIPS                         3142 non-null   int64  
 5   county_pop2018_18 and older  3142 non-null   int64  
 6   anycondition_prevalence      3142 non-null   float64
 7   anycondition_Lower 95% CI    3142 non-null   float64
 8   anycondition_Upper 95% CI    3142 non-null   float64
 9   anycondition_number          3142 non-null   int64  
 10  Obesity_prevalence           3142 non-null   float64
 11  Obesity_Lower 95% CI         3142 non-null   float64
 12  Obesity_Upper 95% CI         3142 non-null   float64
 13  Obesity_number    

In [26]:
df3.head()


,COUNTY_NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,county_pop2018_18 and older,anycondition_prevalence,anycondition_Lower 95% CI,anycondition_Upper 95% CI,anycondition_number,...,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,STAB
0,Autauga,Alabama,1,1,1001,42438,47.6,45.4,49.4,20181,...,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3,AL
1,Baldwin,Alabama,1,3,1003,170912,40.2,38.2,42.3,68790,...,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4,AL
2,Barbour,Alabama,1,5,1005,19689,57.5,55.6,59.1,11325,...,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6,AL
3,Bibb,Alabama,1,7,1007,17813,51.6,49.6,53.4,9190,...,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2,AL
4,Blount,Alabama,1,9,1009,44448,46.3,44.3,48.4,20584,...,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2,AL


In [27]:
cols_drop_df3 = [1,2,3,6,7,8,10,11,12,14,15,16,18,19,20,22,23,24,26,27,28,31]

new_df3 = df3.drop(df3.iloc[:,cols_drop_df3], axis= 1)

In [28]:
# rename FIPS column so it can match with the other datasets

new_df3.rename(columns = {'FIPS':'fips'}, inplace = True)

In [29]:
# merge wids data with medical conditions data

merged_df = new_df.merge(new_df3, on='fips', how='left')

In [30]:
merged_df.head()

,fips,TOT_POP,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,Population Aged 60+,COUNTY_NAME,county_pop2018_18 and older,anycondition_number,Obesity_number,Heart disease_number,COPD_number,diabetes_number,CKD_number,Urban_rural_code
0,1001,55601,6787,7637,6878,7089,7582,7738,5826,4050,...,10523,Autauga,42438,20181,15193,3345,3644,5462,1326,3
1,1003,218022,24757,26913,23579,25213,27338,29986,29932,20936,...,53519,Baldwin,170912,68790,50761,13414,14692,20520,5479,4
2,1005,24881,2732,2960,3268,3201,3074,3278,3076,2244,...,6150,Barbour,19689,11325,8013,2159,2373,3870,887,6
3,1007,22400,2456,2596,3029,3113,3038,3115,2545,1723,...,4773,Bibb,17813,9190,6894,1533,1789,2511,595,2
4,1009,57840,7095,7570,6742,6884,7474,7844,6965,4931,...,13600,Blount,44448,20584,15112,4101,4661,6017,1507,2


In [31]:
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3140 entries, 0 to 3139
Data columns (total 71 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   fips                                                                           3140 non-null   int64  
 1   TOT_POP                                                                        3140 non-null   int64  
 2   0-9                                                                            3140 non-null   int64  
 3   10-19                                                                          3140 non-null   int64  
 4   20-29                                                                          3140 non-null   int64  
 5   30-39                                                                          3140 non-null   int64  
 6   40-49                   

In [32]:
#Merge this new dataset with our previous 'sums' dataframe.

merged_df = merged_df.merge(sums, on='fips', how='left')

In [33]:
merged_df.head()

,fips,TOT_POP,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,county_pop2018_18 and older,anycondition_number,Obesity_number,Heart disease_number,COPD_number,diabetes_number,CKD_number,Urban_rural_code,cases,deaths
0,1001,55601,6787,7637,6878,7089,7582,7738,5826,4050,...,42438,20181,15193,3345,3644,5462,1326,3,1611.0,90.0
1,1003,218022,24757,26913,23579,25213,27338,29986,29932,20936,...,170912,68790,50761,13414,14692,20520,5479,4,5621.0,124.0
2,1005,24881,2732,2960,3268,3201,3074,3278,3076,2244,...,19689,11325,8013,2159,2373,3870,887,6,1193.0,15.0
3,1007,22400,2456,2596,3029,3113,3038,3115,2545,1723,...,17813,9190,6894,1533,1789,2511,595,2,1234.0,6.0
4,1009,57840,7095,7570,6742,6884,7474,7844,6965,4931,...,44448,20584,15112,4101,4661,6017,1507,2,1171.0,0.0


In [34]:
# renaming two columns to see the timeframe

merged_df.rename(columns = {'cases':'cases by May2020', 'deaths':'deaths by May2020'}, inplace = True)

In [35]:
#Save new dataset into interim folder

merged_df.to_csv('../data/interim/merged_dataframe.csv')